In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, max_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data Loading

In [22]:
def load_and_preprocess_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    X = data[['q1', 'q2', 'q3', 'q4', 'q5', 'q6']]
    y = data[['x', 'y', 'z']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

In [23]:
X_train, X_test, y_train, y_test, scaler = load_and_preprocess_data('robot_inverse_kinematics_dataset.csv')


# Data Visualization

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
def create_model():
    model = Sequential([
        Dense(256, activation='relu', input_shape=(6,)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(3)
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [24]:
model1 = create_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 128)                 │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,331 (44.26 KB)

 Trainable params: 11,331 (44.26 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

def calculate_metrics(y_test, y_pred):
    y_test_array = y_test.values if isinstance(y_test, pd.DataFrame) else y_test
    y_pred_array = y_pred.values if isinstance(y_pred, pd.DataFrame) else y_pred

    if y_test_array.ndim > 1: 
        y_pred_flat = y_pred_array.argmax(axis=1) if y_pred_array.ndim > 1 else (y_pred_array > 0.5).astype(int)
        y_test_flat = y_test_array.argmax(axis=1)
    else:  
        y_pred_flat = (y_pred_array > 0.5).astype(int)
        y_test_flat = y_test_array

    accuracy = accuracy_score(y_test_flat, y_pred_flat)
    precision = precision_score(y_test_flat, y_pred_flat, average='weighted')
    recall = recall_score(y_test_flat, y_pred_flat, average='weighted')
    f1 = f1_score(y_test_flat, y_pred_flat, average='weighted')
    cm = confusion_matrix(y_test_flat, y_pred_flat)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm
    }

def train_and_evaluate(model, X_train, X_test, y_train, y_test):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=60,
        batch_size=64,
        validation_split=0.2,
        verbose=1,
        callbacks=[early_stopping]
    )

    y_pred = model.predict(X_test)

    metrics = calculate_metrics(y_test, y_pred)

    return history, metrics, y_pred

history, metrics, y_pred = train_and_evaluate(model1, X_train, X_test, y_train, y_test)

# Print the metrics
print(f"Accuracy: {metrics['accuracy']}")
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")
print(f"F1 Score: {metrics['f1_score']}")
print(f"Confusion Matrix:\n{metrics['confusion_matrix']}")


Epoch 1/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - mae: 0.0289 - val_loss: 6.0314e-04 - val_mae: 0.0189
Epoch 2/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0014 - mae: 0.0289 - val_loss: 6.2467e-04 - val_mae: 0.0192
Epoch 3/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - mae: 0.0293 - val_loss: 5.8601e-04 - val_mae: 0.0187
Epoch 4/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - mae: 0.0290 - val_loss: 6.2930e-04 - val_mae: 0.0194
Epoch 5/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - mae: 0.0288 - val_loss: 5.8930e-04 - val_mae: 0.0187
Epoch 6/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0014 - mae: 0.0288 - val_loss: 5.8664e-04 - val_mae: 0.0188
Epoch 7/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - mae: 0.0288 - val_loss: 5.8740e-04 - val_mae: 0.0185
Epoch 8/60
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0014 - mae: 0.0286 - val_loss: 5.8598e-04 - val_mae: 0.0188
Epoch 9/60
150/150 ━━━━━

In [32]:
model1.save('simple_nn_model.h5')